In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

# Database credentials (Matching your updated etl.py)
user = "root"
password = "" 
host = "localhost"
port = 3307
db = "electronics_dw"

# Create the connection engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}')

# Test connection by listing tables
print("Connected! Available tables:")
print(pd.read_sql("SHOW TABLES", engine))

In [ ]:
# Question A: Average Order Value
query_a = """
SELECT 
    s.Country AS Store_Country,
    f.Order_Year,
    c.Category AS Product_Category,
    AVG(f.Total_Revenue_USD) AS Avg_Order_Value
FROM Sales_Fact f
JOIN Store_Dim s ON f.StoreKey = s.StoreKey
JOIN Product_Dim p ON f.ProductKey = p.ProductKey
JOIN Product_Category_Dim c ON p.SubcategoryKey = c.CategoryKey  -- Adjusting based on your schema
GROUP BY 1, 2, 3
ORDER BY 1, 2, 4 DESC;
"""

df_a = pd.read_sql(query_a, engine)
df_a.head(10)